In [5]:
import numpy as np
import pandas as pd
import psycopg2
from scipy.optimize import fsolve

### Constants

In [6]:
g = 9.81
rho = 1000
k = 0.3 # erodibility constant
b = 1.5 # empirical constant

### Function for wavelength equation

In [7]:
def wavelength_eqn(lambda_, T, d):
    k = 2 * np.pi / lambda_  # Wave number
    return (2 * np.pi / T) ** 2 - g * k * np.tanh(k * d)

In [8]:
def compute_wavelength(T, d):
    deep_water_lambda = (g * T**2) / (2 * np.pi)  # deep water approximation
    shallow_water_lambda = T * np.sqrt(g * d)  # shallow water approximation

    if d > deep_water_lambda / 2:
        return deep_water_lambda  # deep water case
    elif d < shallow_water_lambda / 20:
        return shallow_water_lambda  # shallow water case
    else:
        return fsolve(wavelength_eqn, deep_water_lambda, args=(T, d))[0] 

### Generate synthetic dataset

In [9]:
num_samples = 10000
time_periods = np.random.uniform(1, 7, num_samples)  
depths = np.random.uniform(0.5, 5, num_samples)  
# wave height = 0.78 * depth
wave_heights = np.random.uniform(0.5 * 0.78, 5 * 0.78, num_samples)

In [10]:
wavelengths = []
velocities = []
energies = []
powers = []
shear_stresses = []
impact_pressures = []
soil_erosion_rates = []

In [11]:
for T, d, H in zip(time_periods, depths, wave_heights):
    lambda_value = compute_wavelength(T, d)  # Compute wavelength
    velocity = lambda_value / T  # Wave velocity
    energy = 0.5 * rho * g * H**2  # Wave energy
    power = energy * velocity  # Wave power
    shear_stress = (rho * g * H) / d  # Shear stress
    soil_erosion_rate = k * (shear_stress ** b)  # Soil erosion rate
    impact_pressure = rho * (velocity ** 2)  # Impact pressure

    wavelengths.append(lambda_value)
    velocities.append(velocity)
    energies.append(energy)
    powers.append(power)
    shear_stresses.append(shear_stress)
    soil_erosion_rates.append(soil_erosion_rate)
    impact_pressures.append(impact_pressure)

/tmp/ipykernel_7302/2941639387.py:10: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  return fsolve(wavelength_eqn, deep_water_lambda, args=(T, d))[0]


In [12]:
df = pd.DataFrame({
    'time_period': time_periods,
    'depth': depths,
    'wave_height': wave_heights,
    'wavelength': wavelengths,
    'velocity': velocities,
    'energy': energies,
    'power': powers,
    'shear_stress': shear_stresses,
    'soil_erosion_rate': soil_erosion_rates,
    'impact_pressure': impact_pressures
})

In [13]:
df.head()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
0,4.640135,2.075021,0.648022,19.576206,4.218887,2059.769442,8689.934708,3063.630671,5.087166e+04,17799.008276
1,4.700769,4.755215,1.969944,27.465478,5.842763,19034.724468,111215.380355,4063.990374,7.772313e+04,34137.877339
2,5.620279,3.350083,2.319816,29.917688,5.323168,26396.476609,140512.878257,6793.082926,1.679661e+05,28336.117016
3,5.652439,0.557320,2.449164,13.216686,2.338227,29422.180292,68795.731733,43110.459443,2.685315e+06,5467.304769
4,1.109864,4.677493,0.840049,1.923217,1.732841,3461.373381,5998.009711,1761.816076,2.218514e+04,3002.737931


In [14]:
df.describe()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000
mean,3.982613,2.768668,2.144528,14.084716,3.427321,27642.013098,94763.451805,10622.242593,4.165665e+05,17385.585442
std,1.728937,1.305001,1.018125,14.546504,2.374788,21907.701984,112583.211877,9741.370814,6.280684e+05,10021.734546
min,1.001003,0.500228,0.390077,-38.158877,-5.465323,746.346402,-375537.426474,800.413191,6.793485e+03,2338.734613
25%,2.493709,1.636496,1.262088,6.184292,2.686557,7813.010007,20598.779418,4481.612514,9.000626e+04,8828.696384
50%,3.966647,2.755671,2.152348,14.348968,3.806240,22722.903435,72906.522843,7649.819516,2.007234e+05,15724.699125
75%,5.488541,3.916258,3.032607,23.500584,4.939253,45109.839022,161912.884284,12949.253773,4.420673e+05,24789.167098
max,6.999686,4.999062,3.899465,52.199704,9.024701,74584.601735,478475.420670,71574.573255,5.744595e+06,81445.233690


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time_period        10000 non-null  float64
 1   depth              10000 non-null  float64
 2   wave_height        10000 non-null  float64
 3   wavelength         10000 non-null  float64
 4   velocity           10000 non-null  float64
 5   energy             10000 non-null  float64
 6   power              10000 non-null  float64
 7   shear_stress       10000 non-null  float64
 8   soil_erosion_rate  10000 non-null  float64
 9   impact_pressure    10000 non-null  float64
dtypes: float64(10)
memory usage: 781.4 KB


In [16]:
df = df[(df >= 0).all(axis=1)]

In [17]:
df.head()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
0,4.640135,2.075021,0.648022,19.576206,4.218887,2059.769442,8689.934708,3063.630671,5.087166e+04,17799.008276
1,4.700769,4.755215,1.969944,27.465478,5.842763,19034.724468,111215.380355,4063.990374,7.772313e+04,34137.877339
2,5.620279,3.350083,2.319816,29.917688,5.323168,26396.476609,140512.878257,6793.082926,1.679661e+05,28336.117016
3,5.652439,0.557320,2.449164,13.216686,2.338227,29422.180292,68795.731733,43110.459443,2.685315e+06,5467.304769
4,1.109864,4.677493,0.840049,1.923217,1.732841,3461.373381,5998.009711,1761.816076,2.218514e+04,3002.737931


In [18]:
df.describe()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
count,9320.000000,9320.000000,9320.000000,9320.000000,9320.000000,9320.000000,9320.000000,9320.000000,9.320000e+03,9320.000000
mean,3.869880,2.831667,2.144220,16.813742,3.974433,27628.151129,109952.343351,10434.202641,4.078206e+05,17399.383703
std,1.714294,1.317034,1.017390,10.610231,1.266270,21881.977598,97712.740444,9728.348710,6.285484e+05,10246.393500
min,1.001003,0.500228,0.390077,1.547988,1.529292,746.346402,1374.803556,800.413191,6.793485e+03,2338.734613
25%,2.390503,1.670436,1.259028,7.895972,2.931126,7775.162785,27792.116298,4381.232349,8.699929e+04,8591.498746
50%,3.801168,2.891764,2.159421,15.367593,3.938832,22872.502938,82395.371202,7480.119986,1.940815e+05,15514.397614
75%,5.331213,3.990153,3.029047,24.207463,5.025990,45004.003618,168927.526746,12489.496756,4.187344e+05,25260.571946
max,6.999686,4.999062,3.899465,52.199704,9.024701,74584.601735,478475.420670,71574.573255,5.744595e+06,81445.233690


In [19]:
df.to_csv("synthetic_wave_data.csv", index=False)

In [20]:
df.head()

,time_period,depth,wave_height,wavelength,velocity,energy,power,shear_stress,soil_erosion_rate,impact_pressure
0,4.640135,2.075021,0.648022,19.576206,4.218887,2059.769442,8689.934708,3063.630671,5.087166e+04,17799.008276
1,4.700769,4.755215,1.969944,27.465478,5.842763,19034.724468,111215.380355,4063.990374,7.772313e+04,34137.877339
2,5.620279,3.350083,2.319816,29.917688,5.323168,26396.476609,140512.878257,6793.082926,1.679661e+05,28336.117016
3,5.652439,0.557320,2.449164,13.216686,2.338227,29422.180292,68795.731733,43110.459443,2.685315e+06,5467.304769
4,1.109864,4.677493,0.840049,1.923217,1.732841,3461.373381,5998.009711,1761.816076,2.218514e+04,3002.737931
